In [1]:
import requests
import csv
import time
import pandas as pd
import numpy as np

from datetime import datetime
from google.auth import default
from google.auth.transport.requests import Request

In [2]:
def get_gcloud_credential():
    credentials, project = default()
    if not credentials.valid:
        if credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
    return credentials

In [3]:
def circle_search(credentials, lat, long, radius, types):
    api_url = 'https://places.googleapis.com/v1/places:searchNearby'
    # lat, long = [float(element) for element in coordinate.strip('()').split(',')]
    # lat, long = coordinate[0], coordinate[1]
    field = ('places.displayName,'
             'places.types,'
             'places.priceLevel,'
             'places.rating,'
             'places.photos')
    
    header = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {credentials.token}',
        'X-Goog-FieldMask': field,
        'X-Goog-User-Project': credentials.quota_project_id
    }
    
    payload = {
        'locationRestriction': {
            'circle': {
                'center': {
                    'latitude': lat,
                    'longitude': long
                },
                'radius': radius
            }
        },
        'includedTypes': types
    }
    
    response = requests.post(api_url, headers=header, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print(f'Error: {response.status_code}')
        print(response.json())
        return None

In [4]:
def dataset_maker(data):
    now = datetime.now().strftime('%d-%m-%y_%H%M%S')
    banned_types = ['point_of_interest', 'establishment']
    file_path = f'dataset-output/export_{now}.csv'
    dataset = []
    
    for item in data.get('places', []):
        types = [t for t in item.get('types', []) if t not in banned_types]
        row = {
            'Name': item.get('displayName', {}).get('text', ),
            'Types': ', '.join(types),
            'Price Level': item.get('priceLevel', ),
            'Rating': item.get('rating', ),
            'Photos': item.get('photos', )
        }
        dataset.append(row)
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=dataset[0].keys())
        writer.writeheader()
        writer.writerows(dataset)
    
    print(f'Dataset saved to {file_path}')

In [5]:
# coordination = '-8.67039555301166, 115.21251859347734'
# credential = get_gcloud_credential()
# circle_radius = 500
# 
# places = circle_search(credential, coordination, circle_radius)
# 
# if places:
#     for place in places.get('places', []):
#         display_name = place.get('displayName', 'no displayname found')
#         print(f'Name        : {display_name.get("text", "name not found")}')
#         print(f'Types       : {place.get("types", "types not found")}')
#         print(f'Price Level : {place.get("priceLevel", "no price level found")}')
#         print(f'Rating      : {place.get("rating", "rating not found")}')
#         print('---------------------------------------------------------')
# else:
#     print('No places found')
# 
# dataset_maker(places)

In [6]:
# print(coord['coordinate'][0])

In [7]:
# print(np.array(coord['coordinate'][0][1]))

In [8]:
place_types = [
    'car_rental', 'art_gallery', 'museum', 'performing_arts_theater', 'amusement_park', 'cultural_center',
    'hiking_area', 'historical_landmark', 'national_park', 'night_club', 'park', 'tourist_attraction', 'zoo',
    'american_restaurant', 'bar', 'barbecue_restaurant', 'brazilian_restaurant', 'cafe', 'chinese_restaurant',
    'coffee_shop', 'fast_food_restaurant',
    'french_restaurant', 'greek_restaurant', 'indian_restaurant',
    'indonesian_restaurant', 'italian_restaurant', 'japanese_restaurant', 'korean_restaurant', 'lebanese_restaurant',
    'mediterranean_restaurant', 'mexican_restaurant', 'middle_eastern_restaurant', 'restaurant', 'seafood_restaurant',
    'spanish_restaurant',
    'steak_house', 'sushi_restaurant', 'thai_restaurant', 'turkish_restaurant',
    'vietnamese_restaurant', 'camping_cabin', 'cottage', 'guest_house', 'hostel', 'hotel', 'lodging', 'motel',
    'private_guest_room', 'resort_hotel'
]

print(len(place_types))

49


In [9]:
coord = pd.read_csv('coordinates/filtered_land_points.csv')
radius = 500
credential = get_gcloud_credential()


for i in range(10):
    places = circle_search(credential, coord['lat'][i], coord['lon'][i], radius, place_types)
    dataset_maker(places)
    time.sleep(1)

Dataset saved to dataset-output/export_05-06-24_223916.csv
Dataset saved to dataset-output/export_05-06-24_223918.csv
Dataset saved to dataset-output/export_05-06-24_223919.csv
Dataset saved to dataset-output/export_05-06-24_223921.csv
Dataset saved to dataset-output/export_05-06-24_223922.csv
Dataset saved to dataset-output/export_05-06-24_223923.csv
Dataset saved to dataset-output/export_05-06-24_223924.csv
Dataset saved to dataset-output/export_05-06-24_223926.csv
Dataset saved to dataset-output/export_05-06-24_223927.csv
Dataset saved to dataset-output/export_05-06-24_223928.csv


In [17]:
print(len(coord))

6406


In [10]:
print(f"({coord['lat'][0]}, {coord['lon'][0]})")

(-8.848271756339026, 115.12836912186044)


In [11]:
# credential = get_gcloud_credential()
# coord = '(-8.026006999300794, 114.42196422294008)'
# radius = 500
# 
# places = circle_search(credential, coord, radius, place_types)

In [12]:
# dataset_maker(places)

In [14]:
import os
import pandas as pd
from pandas.errors import EmptyDataError

# Path to the folder containing the CSV files
folder_path = 'dataset-output'

# List to hold dataframes
dataframes = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a CSV
    if filename.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(folder_path, filename)
        try:
            # Read the CSV file
            df = pd.read_csv(file_path)
            # Append the dataframe to the list if it's not empty
            if not df.empty:
                dataframes.append(df)
        except EmptyDataError:
            print(f"Warning: {file_path} is empty and will be skipped.")

# Optionally, concatenate all dataframes into a single dataframe if the list is not empty
if dataframes:
    all_data = pd.concat(dataframes, ignore_index=True)
    # Display the concatenated dataframe
    print(all_data)
else:
    print("No dataframes to concatenate.")


                                        Name  \
0                   Six Senses Uluwatu, Bali   
1                          Oneeighty Dayclub   
2    The cave by Chef Ryan Clift at The edge   
3      Cliff Bar at Six Senses Uluwatu, Bali   
4                              Samsara beach   
..                                       ...   
134             Villa Anasera by Bukit Vista   
135                   Majestic Water Village   
136                         Bale Taru Pecatu   
137                        D’kubu rujakhouse   
138                          Villa Sunseeker   

                                                 Types  \
0                         resort_hotel, hotel, lodging   
1    restaurant, swimming_pool, bar, sports_complex...   
2                                     restaurant, food   
3                                                  bar   
4                                   tourist_attraction   
..                                                 ...   
134              

In [15]:
all_data

,Name,Types,Price Level,Rating,Photos
0,"Six Senses Uluwatu, Bali","resort_hotel, hotel, lodging",NaN,4.6,[{'name': 'places/ChIJhWw69uVa0i0RyDeb1FFs6Lo/...
1,Oneeighty Dayclub,"restaurant, swimming_pool, bar, sports_complex...",NaN,4.6,[{'name': 'places/ChIJ3Tq7gu5a0i0RwfmpF7w_jfk/...
2,The cave by Chef Ryan Clift at The edge,"restaurant, food",PRICE_LEVEL_VERY_EXPENSIVE,4.3,[{'name': 'places/ChIJz3aiexxb0i0RxiMiyHGceqo/...
3,"Cliff Bar at Six Senses Uluwatu, Bali",bar,NaN,4.6,[{'name': 'places/ChIJ8ZOmGRVb0i0R_8BTh60a1g8/...
4,Samsara beach,tourist_attraction,NaN,3.7,[{'name': 'places/ChIJ3Zz7D8hb0i0RjgeiNWFB4IM/...
...,...,...,...,...,...
134,Villa Anasera by Bukit Vista,"hotel, lodging",NaN,4.9,[{'name': 'places/ChIJNYVUtcdb0i0RQH6Syz6Sj2w/...
135,Majestic Water Village,lodging,NaN,4.4,[{'name': 'places/ChIJbdWq2eBa0i0RGfr745BteSE/...
136,Bale Taru Pecatu,"hotel, lodging",NaN,5.0,[{'name': 'places/ChIJ83JuYc5b0i0Ry_zv0wcgwiY/...
137,D’kubu rujakhouse,"restaurant, food",NaN,5.0,[{'name': 'places/ChIJLwX66J1b0i0RswUQBtZH0wU/...
